In [1]:


from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import math
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

In [2]:
driver_path = "chromedriver.exe"  #
brave_path = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.binary_location = brave_path

# 抓內容

In [10]:
# 爬特定出版社書籍


In [76]:
# 爬總排名function
# 如果爬不到會以"_"標記
def FIND_RANK():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    if soup.text.replace("\n", "").strip(" ") == "Page Not Found":
        all_rank = "_"
        # 出現這個表示該頁面不存在
        print("[", ind, "]", "Page Not Found!!!!!!!")

    try:
        ul = soup.find_all("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")
        rank_str = ul[1].find("span", class_="a-list-item").text
        rank_str = rank_str.split(" in", 1)[0].strip("  Best Sellers Rank:  ")
        all_rank = rank_str

    except IndexError:
        try:
            ul = soup.find("table", class_="a-keyvalue a-vertical-stripes a-span6")
            rank_str = ul.find_all("tr")[-1].find_all("span")[1].text.split(" in", 1)[0]
            all_rank = rank_str

        except AttributeError:
            try:
                ul = soup.find_all("table", class_="a-keyvalue prodDetTable")[1]
                rank_str = ul.find_all("tr")[-2].find_all("span")[1].text.split(" in", 1)[0]
                all_rank = rank_str

            except IndexError:
                all_rank = "_"

    except AttributeError:
        try:
            all_rank = "_"
            # 出現這個表示該商品欄位只有評價
            print("[", ind, "]", "detail only has consumer review???????")
        except AttributeError:
            all_rank = "_"

    print("[", ind, "]", df.title[ind])

    return all_rank


# 爬其他資訊
def FIND_OTHER_INFO():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    if soup.text.replace("\n", "").strip(" ") == "Page Not Found":
        other_info = "_"

    try:
        ul = soup.find_all("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")

        detail = soup.find("ul", class_="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list").text
        detail = detail.replace(
            "\n                                    \u200f\n                                        :\n                                    \u200e\n                                 ", ":"
        )
        other_info = detail

    except IndexError:
        try:
            ul = soup.find("table", class_="a-keyvalue a-vertical-stripes a-span6")

            detail = ul.text
            other_info = detail

        except AttributeError:
            try:
                ul = soup.find_all("table", class_="a-keyvalue prodDetTable")[1]

                detail = ul.text
                other_info = detail

            except IndexError:
                other_info = "_"

    except AttributeError:
        other_info = "_"

    return other_info


# 爬主要資訊
def APPEND_LIST():

    df_append = pd.DataFrame({})

    # price
    try:
        p = soup.find("div", class_="a-section a-spacing-none a-text-right")
        p = p.find("span", class_="a-color-secondary a-text-strike")
        before_discount = p.text
    except AttributeError:
        try:
            p = soup.find("tr", class_="celwidget print-list-price")
            before_discount = p.text
        except:
            try:
                p = soup.find("span", class_="aok-relative").find("span", class_="a-offscreen")
                before_discount = p.text
            except:
                before_discount = "_"

    try:
        a_button_inners = MMGridLayout.find_all("span", class_="a-button-inner")
        a_button_inner = a_button_inners[block]
        slot_price = a_button_inner.find("span", class_="slot-price")
        new_price = slot_price.text.replace("\n", "").strip(" ")
    except:
        a_button_inner = MMGridLayout.find("span", class_="a-button-inner")
        slot_price = a_button_inner.find("span", class_="slot-price")
        new_price = slot_price.text.replace("\n", "").strip(" ")

    try:
        extraMessage = a_button_inner.find("span", class_="slot-extraMessage").text
        new_price = new_price + extraMessage
    except:
        extraMessage = ""

    df_append = pd.DataFrame(
        {
            "all_rank_list": [FIND_RANK()],
            "format": [format_str_orign.casefold().capitalize()],
            "before_discount": [before_discount],
            "new_price": [new_price],
            "title": [df.title[ind]],
            "scrape_time": [datetime.now() + timedelta(hours=8)],
            "other_list": [FIND_OTHER_INFO()],
            "department": [str(df.publisher[ind]).replace("['",'').replace("']",'').replace("'","")],
            "author": [str(df.author_name[ind]).replace("['",'').replace("']","")],
        }
    )
    return df_append

def set_up_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    # return webdriver.Chrome(service=Service(r"C:\Users\USER\OneDrive\桌面\C2L2\chromedriver.exe"), options=chrome_options)
    return webdriver.Chrome( options=chrome_options)
    


In [92]:
# 建空的資料夾
Best_Seller_other_info = pd.DataFrame(
    {
        "all_rank_list": [],  # 總排名
        "format": [],  # 書的類別(paperback、hardcover等)
        "before_discount": [],  # 打折前價格(如果沒打折，會標記為"_")
        "new_price": [],  # 新價格
        "title": [],  # 書名
        "scrape_time": [],  # 爬取書名
        "other_list": [],  # 其他資訊
        "department": [],  # 類別
        "author": [],
    }
)  # 書名
# 如果讀不到東西，會用這個存起來
not_append = []

In [93]:
df = pd.read_csv("./HPCL_books_data.csv")
df = df[df['first_publish_year']<=2024]
df = df[df['first_publish_year']>=2000]
df['first_publish_year'] = df['first_publish_year'].astype(int)
samples = []
for i in range(2000,2024):
    samples.append(df[df['first_publish_year'] == i].sample(n = 20,random_state = 1))



In [94]:
# 開啟selenium chrome driver
##############記得要把語言改成英文，不然價格就不是美金
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)


In [103]:
driver = set_up_driver(headless=False)
driver.get("https://www.amazon.com/ref=nav_logo")
nappByYear = []
for year,idf in enumerate(samples): 
    not_append = []
    for ind in idf.index:
        # 搜尋作者
        try:
            author_item = df.author_name[ind].lstrip("['").rstrip("']").replace("', '", " ")
        except AttributeError:
            author_item = ""
        #item = df.title[ind] + " " + author_item
        #item_string = "+".join(item.split())
        driver.get("https://www.amazon.com/advanced-search/books")
        driver.refresh()
        time.sleep(1)
        element = driver.find_elements(By.NAME, "field-keywords")
        element[1].send_keys(df.title[ind])
        element = driver.find_element(By.NAME,"field-author")
        element.send_keys(author_item)
        button = driver.find_element(By.NAME,"Adv-Srch-Books-Submit")
        button.click()
        print(f"{df.title[ind]} : {author_item}")  
        
        try:
            title = driver.find_element(By.CLASS_NAME, "a-size-mini.a-spacing-none.a-color-base.s-line-clamp-2").find_element(By.TAG_NAME, "a")
        except:
            try:
                title = driver.find_element(By.CLASS_NAME, "a-size-mini.a-spacing-none.a-color-base.s-line-clamp-4").find_element(By.TAG_NAME, "a")
            except:
                not_append.append(df.title[ind])
                continue
        title.click()
        time.sleep(1)
        driver.refresh()
    
        i = 0
        while True:
    
            time.sleep(1)
            # 將現在網頁內容裝進python的soup內
            soup = BeautifulSoup(driver.page_source, "html.parser")
            i = i + 1
            print(i, end=" ")
            # 錯誤頁面
            refuse_request = [
                "Request was throttled. Please wait a moment and refresh the page",
                "      503 - Service Unavailable Error",
                "Amazon.comEnter the characters you see belowSorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies.Type the characters you see in this image:Try different imageContinue shoppingConditions of UsePrivacy Policy          © 1996-2014, Amazon.com, Inc. or its affiliates          ",
            ]
            if soup.text.replace("\n", "") in refuse_request:
                driver.refresh()
                time.sleep(30 * i)
    
            else:
                break
    
        # 即使顯示Page Not Found，還是裝進資料，事後檢查
        if soup.text.replace("\n", "").strip(" ") == "Page Not Found":
            df_append = pd.DataFrame(
                {
                    "all_rank_list": ["_"],
                    "format": [df.format[ind]],
                    "before_discount": ["_"],
                    "new_price": [df.price[ind]],
                    "title": [df.title[ind]],
                    "scrape_time": [datetime.now() + timedelta(hours=8)],
                    "other_list": ["_"],
                    "department": [df.department[ind]],
                    "author": [df.author[ind]],
                }
            )
            Best_Seller_other_info = pd.concat([Best_Seller_other_info, df_append], ignore_index=True)
    
        # 爬取商品頁面右上角框框
        MMGridLayout = soup.find("div", class_="a-section a-spacing-none MMGridLayout")
        price_box = str(MMGridLayout).count("a-button-inner")
        if price_box > 1:
    
            for block in range(price_box):
                soup = BeautifulSoup(driver.page_source, "html.parser")
                MMGridLayout = soup.find("div", class_="a-section a-spacing-none MMGridLayout")
                try:
                    a_button_inner = MMGridLayout.find_all("span", class_="a-button-inner")[block]
                except:
                    try:
                        time.sleep(5)
                        a_button_inner = MMGridLayout.find_all("span", class_="a-button-inner")[block]
                    except AttributeError:
                        not_append.append(f"{df.title[ind]}")
                        driver.back()
                        time.sleep(1)
                        continue
                format_str = a_button_inner.find("span", class_="slot-title").text.replace("\n", "").strip(" ").upper().replace("-", "_").replace(" ", "_").replace("\xa0", "")
                try:
                    format_str = format_str.split("___")[0]
                except:
                    format_str = format_str
    
                format_str_orign = format_str
    
                # selenium需要python這邊的字串和HTML格式相符才會點進網頁
                paperback = ["PERFECT_PAPERBACK", "PAMPHLET", "UNBOUND", "PAPERBACK_BUNKO"]
                if format_str in paperback:
                    format_str = "PAPERBACK"
    
                hardcover = [
                    "LIBRARY_BINDING",
                    "LEATHER_BOUND",
                    "PRINTED_ACCESS_CODE",
                    "IMITATION_LEATHER",
                    "SCHOOL_&_LIBRARY_BINDING",
                    "PRODUCT_BUNDLE",
                    "BONDED_LEATHER",
                    "VINYL_BOUND",
                    "HARDCOVER_SPIRAL",
                    "TURTLEBACK",
                ]
                if format_str in hardcover:
                    format_str = "HARDCOVER"
    
                if format_str == "AUDIOBOOK":
                    format_str = "AUDIO_DOWNLOAD"
    
                audiobook = ["AUDIO_CD", "MP3_CD", "PRELOADED_DIGITAL_AUDIO_PLAYER", "AUDIO,_CASSETTE", "MP3_CD_LIBRARY_BINDING"]
                if format_str in audiobook:
                    format_str = "AUDIOBOOK"
    
                kindle = ["KINDLE_EDITION_WITH_AUDIO/VIDEO", "ETEXTBOOK"]
                if format_str in kindle:
                    format_str = "KINDLE"
    
                if format_str == "LOOSE_LEAF":
                    format_str = "LOOSELEAF"
    
                other_format = [
                    "BOARD_BOOK",
                    "FLEXIBOUND",
                    "MULTIMEDIA_CD",
                    "MAGAZINE",
                    "SHEET_MUSIC",
                    "CARDS",
                    "COMICS",
                    "CALENDAR",
                    "POCKET_BOOK",
                    "DIGITAL",
                    "GAME",
                    "MAP",
                    "WALL_CHART",
                    "BOOK_SUPPLEMENT",
                    "TOY",
                    "BATH_BOOK",
                    "POCKET_BOOK",
                    "TEXTBOOK_BINDING",
                    "WALL_CHART",
                    "PLASTIC_COMB",
                ]
                if format_str in other_format:
                    format_str = "OTHER"
    
                # 點進該書之各個format之商品頁面
                try:
                    driver.find_element(By.ID, f"tmm-grid-swatch-{format_str}").click()
                except:
                    not_append.append(f"{df.title[ind]}: {format_str}")
                    continue
    
                time.sleep(1)
                soup = BeautifulSoup(driver.page_source, "html.parser")
                if soup.text.replace("\n", "") in refuse_request:
                    i = 1
                    while True:
                        print(i, end=" ")
                        driver.refresh()
                        time.sleep(1 + (3 * i))
                        soup = BeautifulSoup(driver.page_source, "html.parser")
                        i = i + 1
                        if soup.text.replace("\n", "") not in refuse_request:
                            break
    
                df_append = APPEND_LIST()
                Best_Seller_other_info = pd.concat([Best_Seller_other_info, df_append], ignore_index=True)
    
        else:
            # 如果只有一種類別
            try:
                a_button_inner = MMGridLayout.find("span", class_="a-button-inner")
                format_str = a_button_inner.find("span", class_="slot-title").text.replace("\n", "").strip(" ").upper().replace("-", "_").replace(" ", "_").replace("\xa0", "")
                try:
                    format_str = format_str.split("___")[0]
                except:
                    format_str = format_str
    
                format_str_orign = format_str
                df_append = APPEND_LIST()
                Best_Seller_other_info = pd.concat([Best_Seller_other_info, df_append], ignore_index=True)
    
            # 商品頁面右上角沒有框框的時候(不另外寫function)
            except AttributeError:
    
                try:
                    before_discount = soup.find("span", class_="a-price a-text-price").find("span", class_="a-offscreen").text
                except AttributeError:
                    before_discount = "_"
    
                try:
                    new_price = soup.find("span", class_="a-price aok-align-center").find("span", class_="a-offscreen").text
                except AttributeError:
                    try:
                        new_price = soup.find("span", class_="a-price a-text-normal aok-align-center reinventPriceAccordionT2").find("span", class_="a-offscreen").text
                    except AttributeError:
                        new_price = "_"
                    
    
                try:
                    extraMessage = a_button_inner.find("span", class_="slot-extraMessage").text
                    new_price = new_price + extraMessage
                except:
                    extraMessage = ""
    
                df_append = pd.DataFrame(
                    {
                        "all_rank_list": [FIND_RANK()],
                        "format": ['_'],
                        "before_discount": [before_discount],
                        "new_price": [new_price],
                        "title": [df.title[ind]],
                        "scrape_time": [datetime.now() + timedelta(hours=8)],
                        "other_list": [FIND_OTHER_INFO()],
                        "department": [""],
                        "author": [""],
                    }
                )
                Best_Seller_other_info = pd.concat([Best_Seller_other_info, df_append], ignore_index=True)
    
        print("[", ind, "]", df.title[ind])
    nappByYear.append([year+1999,len(not_append)])
    # 關掉selenium chrome driver
    driver.quit()
print(nappByYear)

Sport and Racing Cars : R. F. Yates
Destination : Seymour Simon
The Black Brook : Tom Drury
Complete Christian Classics : Owen Collins
Throwing smoke : Bruce Brooks
A pie went by : Carolyn Dunn
Auctioneer : Charles Fernyhough
Little League Visitor : Bishop                       C
The Vegetarian Society's the new vegetarian cookbook : Heather Thomas
Think & play winning rugby : Hugh de Lacy Grant Fox
Letters from Vietnam : Morley Safer
Ice Trek : Eric Philips
Two Princesses Of Bamarre : Gail Carson Levine January Lavoy Lynn Redgrave
Among Insurgents : Shelby Tucker
Coriolanus : Hudson
The gluten, wheat & dairy free cookbook : Antoinette Savill
Journey from Abandonment to Healing : Susan Andersen
Living Beau. 18c : A. Stoddard
World's Great Bridges : Shirley-Smith                H
Life Is So Good : George Dawson Richard Glaubman


MaxRetryError: HTTPConnectionPool(host='localhost', port=64081): Max retries exceeded with url: /session/145ec9b20c7268e9e19d07144497655b/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15f3101f0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [101]:
nappByYear

[]

In [ ]:
# 儲存
Best_Seller_other_info.to_csv("HarperCollins_Amazon.csv", index=False)

# 存沒爬到的東西
with open("not_append.txt", "w") as f:
    for row in not_append:
        f.write("%s\n" % row)

In [ ]:
df

In [ ]:
# 檢查沒爬到的書
for ind in Best_Seller_other_info.index:
    if Best_Seller_other_info["all_rank_list"] == "_":
        print(ind, Best_Seller_other_info["title"])